In [31]:
import pandas as pd
import numpy as np
import os 

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [32]:
input_path = os.path.join('..\data\datasets\csv_files','2016-2020-v2.csv')
df = pd.read_csv(input_path)

In [33]:
df.columns

Index(['period', 'period_type', 'period_time', 'game_seconds', 'gameID',
       'attacking_team_id', 'attacking_team_name', 'home_team', 'shooter',
       'goalie', 'shot_type', 'x_coordinate', 'y_coordinate', 'strength',
       'last_event_type', 'last_event_x', 'last_event_y',
       'time_since_last_event', 'distance_from_last_event',
       'powerplay_duration', 'home_team_players', 'away_team_players',
       'distance_to_net', 'shot_angle', 'is_goal', 'is_empty_net', 'rebound',
       'angle_change', 'speed'],
      dtype='object')

$$ \textbf{Traitement des valeurs manquantes dans la colonne 'strength':} $$

In [3]:
def remove_nan_from_strength(df : pd.DataFrame):
    for index, row in df.iterrows():
    # On cherche les lignes pour lesquelles 
        if (row['strength'] != 'Even') & (row['strength'] != 'Power Play') & (row['strength'] != 'Short Handed'):
            if row['attacking_team_name'] == row['home_team'] :
                if row['home_team_players'] < row['away_team_players']:
                    df.at[index, 'strength'] = 'Short Handed'
                else : 
                    if row['home_team_players'] == row['away_team_players']:
                        df.at[index, 'strength'] = 'Even'
                    else :
                        df.at[index, 'strength'] = 'Power Play'
            else :
                if row['away_team_players'] > row['home_team_players']:
                    df.at[index, 'strength'] = 'Power Play'
                else : 
                    if row['away_team_players'] == row['home_team_players']:
                        df.at[index, 'strength'] = 'Even'
                    else : 
                        df.at[index, 'strength'] = 'Short Handed'

    return df

In [4]:
df = remove_nan_from_strength(df)

$$ \textbf{Encodage des caractéristiques : } $$

In [5]:
# Encodage des caractéristiques de type catégorielle :
# On utilise LabelEncoder() pour les variables pour lesquelles l'ordre n'est pas important

# Colonne pour lesquels l'ordre n'est pas important
categorical_columns_1 = ['period_type', 'attacking_team_name', 'shooter', 'goalie', 'rebound', 'last_event_type', 'home_team']

# Colonne pour laquelle l'ordre est important
# Sachant que certains types de tirs sont plus efficaces en moyenne que d'autre, on encode les 
# types de tirs les plus efficaces avec des valeurs élevées
# (Au Milestone 1, on a vu que les 'Tip-in' sont les plus efficaces et que les 'Wrap-around' sont les moins
# efficaces)

shot_type_classified = [['Wrap-around',0], ['Slap Shot', 1], ['Snap Shot', 2], ['Wrist Shot', 3], ['Backhand', 4], ['Deflected', 5], ['Tip-In',6]]

# La caractéristique 'strength' doit aussi étre encodée de manière ordinale, étant donné
# que lorsqu'une équipe est en 'Power Play', elle a plus de chances de marquer tandis que lorsqu'elle est
# 'Short handed', ses chances de marquer diminuent
strength_classified = [['Short Handed',0], ['Even', 1], ['Power Play', 2]]

In [6]:
def encode_categorical_features(df : pd.DataFrame, categorical_features: list, shot_type_classified : list, strength_classified : list):
    df = df.copy()

    # Encodage des 'shot_type'
    mapping_dict = {row[0]: row[1] for row in shot_type_classified}
    df['shot_type'] = df['shot_type'].replace(mapping_dict)

    # Encodage de 'strength'
    mapping_dict_1 = {row[0] : row[1] for row in strength_classified}
    df['strength'] = df['strength'].replace(mapping_dict_1)

    # Encodage des autres caractéristiques
    label_encoder = LabelEncoder()

    for feature in categorical_features :
        df[feature] = label_encoder.fit_transform(df[feature]) 

    return df

In [7]:
df = df.dropna()
df = encode_categorical_features(df, categorical_columns_1, shot_type_classified, strength_classified)

$$ \textbf{Sélection des caractéristiques + Séparation des données (entrainement, validation, test):} $$

$$\textbf{Méthode de filtrage (K-best) } $$

In [8]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import recall_score, f1_score

In [9]:
def get_features_KBest(X: pd.DataFrame, Y: pd.Series, nb_features: int):

    # https://stackoverflow.com/questions/39839112/the-easiest-way-for-getting-feature-names-after-running-selectkbest-in-scikit-le
    selector = SelectKBest(f_classif, k = nb_features)
    X_new = pd.DataFrame(selector.fit_transform(X, Y))

    names = X.columns.values[selector.get_support()]

    return X[names], names

In [10]:
def get_test_df(df: pd.DataFrame, test_year : int) :
    df = df.copy()
    # Ajout d'une colonne pour l'année
    df['year'] = df['gameID'].apply(lambda x : x//1000000)

    # Récupération du DataFrame de test
    test_df = df[df['year'] == test_year]
    test_df.drop(columns = 'year')

    # Récupération du DataFrame d'entrainement et validation
    train_val_df = df[df['year'] == test_year]
    train_val_df.drop(columns = 'year')

    return test_df, train_val_df

In [11]:
# Récupération des deux DataFrames
test_df, train_val_df = get_test_df(df, 2020)

In [12]:
X = train_val_df.drop(columns=['is_goal', 'period_time'])
Y = train_val_df['is_goal']

# On récupère le dataset avec les K-meilleures caractéristiques
X_Kbest, Kbest_features = get_features_KBest(X, Y, 10)

c:\Users\yanis\OneDrive\Bureau\UdeM\Science des données\IFT6758-Projet\project-env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [23 27] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\yanis\OneDrive\Bureau\UdeM\Science des données\IFT6758-Projet\project-env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [35]:
X_Kbest.to_csv('../data/datasets/csv_files/Dataset_train.csv', index=False)

In [36]:
Y.to_csv('../data/datasets/csv_files/test_feature_train.csv', index = False)

In [13]:
# Mise à jour de notre ensemble de test
test_df = test_df[Kbest_features]

In [14]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_Kbest, Y, train_size = 0.8, random_state = 42 
)

$$ \textbf{Réequilibrage des données : }  $$ 

In [15]:
# 1re approche : Utilisation de SMOTE pour créer des échantillons synthétiques de la classe minoritaire
from imblearn.over_sampling import SMOTE

In [16]:
oversample_SM = SMOTE(sampling_strategy = 'minority')
X_train_over, Y_train_over = oversample_SM.fit_resample(X_train,Y_train) 

In [17]:
# 2e approche : Utilisation de RandomUnderSampler pour réduire la taille de la classe majoritaire
from imblearn.under_sampling import RandomUnderSampler
# https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler

In [18]:
rus = RandomUnderSampler(random_state = 42)
X_train_res, Y_train_res = rus.fit_resample(X_train, Y_train)

$$ \textbf{Entrainement d'un modèle RandomForest} $$ 

In [70]:
RandomForest_model = RandomForestClassifier(random_state = 42)

In [71]:
# Entrainement du modèle sur les données sur-échantillonées par SMOTE
RandomForest_model.fit(X_train_over,Y_train_over)

RandomForestClassifier(random_state=42)

In [72]:
Y_pred = RandomForest_model.predict(X_val)

In [73]:
confusion_matrix(Y_val,Y_pred)

array([[9111,  630],
       [ 812,  133]], dtype=int64)

In [75]:
print("Accuracy_score :",accuracy_score(Y_pred, Y_val))
print("Recall score :" , recall_score(Y_pred, Y_val))
print("F1-score :" , f1_score(Y_pred, Y_val, average = 'weighted'))

Accuracy_score : 0.8650570840351862
Recall score : 0.1743119266055046
F1-score : 0.8716221840656643


$$ \textbf{Méthode de wrapping : Recursive feature elimination (RFE)} $$

In [58]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

In [59]:
X_1 = train_val_df.drop(columns=['is_goal', 'period_time'])
Y_1 = train_val_df['is_goal']

In [60]:
estimator = SVC(kernel = 'linear')
selector = RFE(estimator, n_features_to_select = 10, step = 1)

selector = selector.fit(X_1,Y_1)

In [61]:
selector.get_feature_names_out()

array(['game_seconds', 'gameID', 'shooter', 'goalie', 'last_event_x',
       'time_since_last_event', 'distance_from_last_event',
       'powerplay_duration', 'distance_to_net', 'speed'], dtype=object)

$$ \textbf{Optimisation des hyperparamètres du modèle : Cross-Validation} $$ 

In [40]:
from sklearn.model_selection import RandomizedSearchCV 

In [41]:
from scipy.stats import randint

In [77]:
param_dist = {'n_estimators' : [100, 150, 200, 250, 300, 400, 500],
              'max_depth' : [5, 10, 15, 20, 30, 40]}

rf = RandomForestClassifier(random_state = 42)

rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, scoring='roc_auc', n_iter = 10, cv = 5)

rand_search.fit(X_train_over, Y_train_over)

best_rf = rand_search.best_estimator_
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'n_estimators': 300, 'max_depth': 30}


In [88]:
rf_opti = RandomForestClassifier(n_estimators = 300, max_depth = 30, random_state = 42)

rf_opti.fit(X_Kbest,Y)

RandomForestClassifier(max_depth=30, n_estimators=300, random_state=42)

In [89]:
Y_pred = rf_opti.predict(X_val)

In [90]:
from sklearn.metrics import roc_auc_score

In [91]:
print("Accuracy_score :",accuracy_score(Y_pred, Y_val))
print("Recall score :" , recall_score(Y_pred, Y_val))
print("F1-score :" , f1_score(Y_pred, Y_val))
print("Roc_auc :", roc_auc_score(Y_pred, Y_val))

Accuracy_score : 0.9992513569155905
Recall score : 1.0
F1-score : 0.9957492029755579
Roc_auc : 0.999589701507847


In [92]:
confusion_matrix(Y_pred, Y_val)

array([[9741,    8],
       [   0,  937]], dtype=int64)

In [44]:
from sklearn.ensemble import AdaBoostClassifier

In [83]:
Ada_clf = AdaBoostClassifier(n_estimators = 100, random_state = 42)

Ada_clf.fit(X_Kbest, Y)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [84]:
Y_pred = Ada_clf.predict(X_val)

In [86]:
print("Accuracy_score :",accuracy_score(Y_pred, Y_val))
print("Recall score :" , recall_score(Y_pred, Y_val))
print("F1-score :" , f1_score(Y_pred, Y_val))

Accuracy_score : 0.9114729552685757
Recall score : 0.3333333333333333
F1-score : 0.0021097046413502112


In [87]:
confusion_matrix(Y_val, Y_pred)

array([[9739,    2],
       [ 944,    1]], dtype=int64)